<a href="https://colab.research.google.com/github/DeepthiTabithaBennet/Python_AppliedStatistics/blob/main/Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Diabetes

`Diabetes.csv` and is [from Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database). We have several questions - what information is more correlated with a positive diagnosis, and if we can only ask two questions to a patient, what should we ask and how would we give them a risk of being diagnosed.

This is a machine learning database, and normally we'd just extract features, feed to a ML algorithm and sit back and relax. But we'll get our hands dirty so that you can learn more.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df_original = pd.read_csv("Diabetes.csv")
df_original.head()

In [ ]:
df = df_original.drop(["Pregnancies", "DiabetesPedigreeFunction"], axis=1)
pd.plotting.scatter_matrix(df, figsize=(6,6), s=2);

Let first answer the question: *What information is most correlated with diabetes?*

In [ ]:
df.corr()

In [ ]:
import seaborn as sb
sb.heatmap(df.corr(), annot=True, cmap="viridis");

From the correlations above, a naive approach might be to take the top correlated observables and drill down into them. In our case, its Glucose, BMI and Age.

In [ ]:
df2 = df[["Glucose", "BMI", "Age", "Outcome"]]
df2.head()

In [ ]:
pd.plotting.scatter_matrix(df2, s=4);

Lucky we visualised this! Look at the histograms for Glucose and BMI - the spikes at zero are indicative that the dataset is using the value zero for when there is no data. Lets filter these out. And lets also drop age to keep this shorter.

In [ ]:
df3 = df2.loc[(df2["Glucose"] > 1) & (df2["BMI"] > 1), ["Glucose", "BMI", "Outcome"]]
df3.head()

In [ ]:
df_y = df3.loc[df3["Outcome"] == 1, ["Glucose", "BMI"]]
df_n = df3.loc[df3["Outcome"] == 0, ["Glucose", "BMI"]]

plt.scatter(df_y["Glucose"], df_y["BMI"], c="w", s=1, label="Has Diabetes")
plt.scatter(df_n["Glucose"], df_n["BMI"], s=1, label="No Diabetes")
plt.legend(loc=2)
plt.xlabel("Glucose")
plt.ylabel("BMI");

In [ ]:
pd.plotting.scatter_matrix(df_y)
pd.plotting.scatter_matrix(df_n);

So its not perfect, but we can probably do an alright job approximating both these distributions as Gaussians.

Let's also add into the mix a test patient that comes in, with Glucose of 140 and BMI of 35. What is his chance of being diagnosed?

In [ ]:
from scipy.stats import multivariate_normal as mn
prob_test = []
test_point = [140, 35]
x, y = np.meshgrid(np.linspace(50, 200, 20), np.linspace(10, 50, 20), indexing='ij')
points = np.dstack((x.flatten(), y.flatten()))

In [ ]:
for d, l in zip([df_y, df_n], ["Yes", "No"]):
    is_yes = l == "Yes"
    mean = np.mean(d)
    cov = np.cov(d, rowvar=0)
    probs = mn.pdf(points, mean, cov).reshape(x.shape)
    prob_test.append(mn.pdf(test_point, mean, cov))
    plt.contour(x, y, probs, 
                cmap="viridis" if is_yes else "magma", 
                linestyles="-" if is_yes else "--")
    marker = "." if is_yes else "+"
    color = "g" if is_yes else "y"
    plt.scatter(d["Glucose"], d["BMI"], c=color, marker=marker, s=20, alpha=0.3, label=l)
plt.axvline(test_point[0], ls=":", lw=1)
plt.axhline(test_point[1], ls=":", lw=1)
plt.legend(loc=2)
plt.xlabel("Glucose")
plt.ylabel("BMI");

In [ ]:
num_y = df_y.shape[0]
num_n = df_n.shape[0]

prob_diagnosis = num_y * prob_test[0] / (num_y * prob_test[0] + num_n * prob_test[1])
print(f"Positive diagnosis chance is {100 * prob_diagnosis:.2f}%")

This might seem odd. Given the test patient is right on the maximum of our model for the diabetes patients, surely there should be a larger chance, right? Nope!

The reason is because - even though the distribution probability is higher, there are far more patients without diabetes than with. We can only directly compare the two distributions if they have equal probability all up (same number of people with and without). This is rarely the case, and so we have to weight them. In a Bayesian formalise, we are modifying our model prior.